### Пример простой глубокой сети прямого распространения для классификации MNIST

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('/tmp/data/', one_hot = True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
n_nodes_1 = 500
n_nodes_2 = 500
n_nodes_3 = 500
n_nodes_4 = 500

In [5]:
n_classes = 10
batch_size = 100

In [6]:
# hight * width
x = tf.placeholder('float',[None, 784])
y = tf.placeholder('float')

In [7]:
def NN_model(data):
    # imput_data * weights + biases
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([784, n_nodes_1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_1]))}
    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_1, n_nodes_2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_2]))}
    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_2, n_nodes_3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_3]))}
    hidden_4_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_3, n_nodes_4])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_4]))}
    output_layer =   {'weights':tf.Variable(tf.random_normal([n_nodes_4, n_classes])),
                      'biases':tf.Variable(tf.random_normal([n_classes]))}
    
    # layer1 = relu(imput_data * weights_1 + biases_1)
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)
    
    # layer2 = relu(layer_1 * weights_2 + biases_2)
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)
    
    # layer3 = relu(layer_2 * weights_3 + biases_3)
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    
    # layer4 = relu(layer_3 * weights_4 + biases_4)
    l4 = tf.add(tf.matmul(l3, hidden_4_layer['weights']), hidden_4_layer['biases'])
    l4 = tf.nn.relu(l4)
    
    # output = layer_4 * weights_output + biases_output
    output = tf.matmul(l4, output_layer['weights']) + output_layer['biases']
    
    return output
    

In [9]:
def train_NN(x):
    predict = NN_model(x)
    # softmax нормализует полученные веса output слоя(сумма всех значений = 1)
    # cross_entropy_with_logits высчитывает ошибку для многоклассовой классификации
    # ошибка вычисляется как distance(softmax(outputs), Labels)) = -summ(Labels * log(outputs))
    # нaм нужно минимизировать эту ошибку
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = predict, labels = y))
    
    # оптимизатор(улучшенный стохастический градиентный спуск)
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    epochs = 10
    
    with tf.Session() as session:
        # инициализируем граф со всеми переменными
        session.run(tf.initialize_all_variables())
        
        for epoch in range(epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                # на каждой эпохе оптимизируем ошибку для каждой части выборки 
                _, c = session.run([optimizer, cost], feed_dict={x:epoch_x, y:epoch_y})
                epoch_loss += c
            print('epoch', epoch, 'complited out of', epochs, 'loss:', epoch_loss)
       
        # считаем метрику accuracy
        correct = tf.equal(tf.argmax(predict, 1), tf.argmax(y, 1))
        
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        print( 'accuracy:', accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))
            
train_NN(x)

epoch 0 complited out of 10 loss: 22836060.9495
epoch 1 complited out of 10 loss: 5084382.87796
epoch 2 complited out of 10 loss: 2607052.54524
epoch 3 complited out of 10 loss: 1517921.46337
epoch 4 complited out of 10 loss: 898453.53627
epoch 5 complited out of 10 loss: 584256.217864
epoch 6 complited out of 10 loss: 403822.820334
epoch 7 complited out of 10 loss: 355662.847492
epoch 8 complited out of 10 loss: 337601.965196
epoch 9 complited out of 10 loss: 345607.649585
accuracy: 0.951
